In [0]:
storage_account = "sanformula1dl"
container_name = "demo,raw,processed,presentation".split(",")
client_id = dbutils.secrets.get(scope="scope-sanformula1dl-access-token-key", key="sanformula1dl-service-account-client-id")
tenant_id = dbutils.secrets.get(scope="scope-sanformula1dl-access-token-key", key="sanformula1dl-service-account-tenant-id")
client_secret_value = dbutils.secrets.get(scope="scope-sanformula1dl-access-token-key", key="sanformula1dl-service-account-client-secret")

In [0]:
config = {
    f"fs.azure.account.auth.type": "OAuth",
    f"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    f"fs.azure.account.oauth2.client.id": client_id,
    f"fs.azure.account.oauth2.client.secret": client_secret_value,
    f"fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
}

In [0]:
# Mount demo
dbutils.fs.mount(
    source = f"abfss://demo@sanformula1dl.dfs.core.windows.net",
    mount_point=f"/mnt/{storage_account}/{container_name[0]}",
    extra_configs=config
)

In [0]:
# Create a function to create mounts
def mount_adls(storage_account_, container_names):
    config_ = {
        f"fs.azure.account.auth.type": "OAuth",
        f"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
        f"fs.azure.account.oauth2.client.id": client_id,
        f"fs.azure.account.oauth2.client.secret": client_secret_value,
        f"fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
    }
    # unmount any existing container_name mounts...
    # for container in container_name_:
    #     for mount in dbutils.fs.mounts():
    #         if mount.mountPoint == f"/mnt/{storage_account_}/{container}":
    #             dbutils.fs.unmount(f"/mnt/{storage_account_}/{container}")
    for container in container_names:
        if any(mount.mountPoint == f"/mnt/{storage_account_}/{container}" for mount in dbutils.fs.mounts()):
            dbutils.fs.unmount(f"/mnt/{storage_account_}/{container}")
            print(f"Unmounted: /mnt/{storage_account_}/{container}")
    for container in container_names:
        try:
            dbutils.fs.mount(
                source = f"abfss://{container}@{storage_account_}.dfs.core.windows.net",
                mount_point=f"/mnt/{storage_account_}/{container}",
                extra_configs=config_
                )
            print(f"Mounted {container} to /mnt/{storage_account_}/{container}")
        except Exception as e:
            print(f"Error mounting {container}: {e}")


In [0]:
mount_adls(storage_account_=storage_account, container_names=container_name)

In [0]:
dbutils.fs.ls("/mnt/demo")

In [0]:
# unmount again
dbutils.fs.unmount("/mnt/demo")

In [0]:
# Create all container mounts
for container in container_name:
    mount_adls(storage_account_=storage_account, container_name_=container)

In [0]:
display(dbutils.fs.mounts())

In [0]:
display(dbutils.fs.ls("/mnt/sanformula1dl/"))